# Imports 

In [13]:
import cv2
import os
import shutil
import numpy as np
import tensorflow as tf
from core.config import cfg
import core.utils as utils
import argparse

# Get Model 

In [14]:
model = tf.keras.models.load_model('SavedModel/YOLOv3_model_v2')
model.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 416, 416, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 416, 416, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [15]:
def read_class_names(class_file_name):
    '''loads class name from a file'''
    names = {}
    with open(class_file_name, 'r') as data:
        for ID, name in enumerate(data):
            names[ID] = name.strip('\n')
    return names

In [16]:
classes = read_class_names(cfg.YOLO.CLASSES)
classes

{0: 'bottom', 1: 'top', 2: 'dress'}

In [78]:
map_classes = {
    "CEKET": "top",
    "DENIM ETEK": "bottom", 
    "DENIM GOMLEK UZUNKOL": "top", 
    "DOKUMA ETEK": "bottom", 
    "GOMLEK KISAKOL": "top", 
    "PANTOLON": "bottom", 
    "T SHIRT": "top",
    "TAKIM ELBISE": "bottom"
} # takım elbise bottom olarak işaretli 

In [47]:
# bboxes: [x_min, y_min, x_max, y_max, probability, cls_id] format coordinates.
# class_ind = int(bbox[5]) 
# classes[class_ind]
def inference(model: tf.keras.models.Model, path_to_image: str, dataset_name: str, cls: str, config: cfg, SCORE_THRESHOLD = 0.7, IOU_THRESHOLD = 0.5): 
    INPUT_SIZE = config.TEST.INPUT_SIZE

    image = cv2.imread(path_to_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_size = image.shape[:2]
    image_data = utils.image_preporcess(np.copy(image), [INPUT_SIZE, INPUT_SIZE])
    image_data = image_data[np.newaxis, ...].astype(np.float32) # (1, width, height, 3)

    pred_bbox = model.predict(image_data)
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    bboxes = utils.postprocess_boxes(pred_bbox, image_size, INPUT_SIZE, SCORE_THRESHOLD)
    bboxes = utils.nms(bboxes, IOU_THRESHOLD, method='nms')

    image_with_detections = utils.draw_bbox(image, bboxes)
    image_with_detections = cv2.cvtColor(image_with_detections, cv2.COLOR_RGB2BGR)

    # If image size is too big then resize the image for display purposes
    display_shape = (image_size[1], image_size[0])
    if image_size[0] > 1500 or image_size[1] > 1500:
        display_shape = (int(image_size[1]/4), int(image_size[0]/4))

    image_with_detections = cv2.resize(image_with_detections, display_shape)

    #cv2.imshow('detections', image_with_detections)
    #cv2.waitKey(0)
    ds_dir = os.path.join("./custom_detections", dataset_name) 
    if not os.path.isdir(ds_dir):
        os.mkdir(ds_dir)
    
    ds_dir = os.path.join(ds_dir, cls)
    if not os.path.isdir(ds_dir):
        os.mkdir(ds_dir)

    return cv2.imwrite(os.path.join(ds_dir, os.path.split(path_to_image)[1]), image_with_detections) 
    

In [57]:
inference(
    model=model, 
    dataset_name="deneme",
    path_to_image="C:/Users/melih.kacaman/Downloads/dennnn.jpg", 
    cls="kafana",
    config=cfg, 
    SCORE_THRESHOLD=0.1
)

1/1 [==============================] - 0s 374ms/step


True

# Get the Test Dataset 

In [20]:
import pandas as pd 

test_df = pd.read_csv("./data/dataset/custom_test_dataset.csv", names=["resim_adresi", "sidsecenek", "UrunGrubuAck", "UrunGrubuKodu"]) 
test_df.head() 

,resim_adresi,sidsecenek,UrunGrubuAck,UrunGrubuKodu
0,https://resim.aydinli.com.tr/02/1/002/S5022910...,696256,CEKET,002
1,https://showroom.aydinli.com.tr/2022K/05/50255...,772080,CEKET,002
2,https://showroom.aydinli.com.tr/2022K/05/50255...,772222,PANTOLON,003
3,https://resim.aydinli.com.tr/02/1/001/S5022929...,697046,TAKIM ELBISE,001
4,https://showroom.aydinli.com.tr/2022K/05/50255...,772044,CEKET,002


In [21]:
test_actual = pd.concat([
    test_df[test_df["UrunGrubuAck"] == "T SHIRT"].iloc[:100, :], 
    test_df[test_df["UrunGrubuAck"] == "CEKET"].iloc[:100, :], 
    test_df[test_df["UrunGrubuAck"] == "TAKIM ELBISE"].iloc[:100, :],
    test_df[test_df["UrunGrubuAck"] == "PANTOLON"].iloc[:100, :], 
    test_df[test_df["UrunGrubuAck"] == "GOMLEK KISAKOL"].iloc[:60, :], 
    test_df[test_df["UrunGrubuAck"] == "DENIM GOMLEK UZUNKOL"].iloc[:40, :],
    test_df[test_df["UrunGrubuAck"] == "DOKUMA ETEK"].iloc[:60, :],
    test_df[test_df["UrunGrubuAck"] == "DENIM ETEK"].iloc[:40, :],
]).reset_index(drop=True) 

In [22]:
import requests
import os

def save_image(image_url, path):
    img_data = requests.get(image_url).content
    with open(path, 'wb') as handler:
        handler.write(img_data)


def save_image_cv(url, scale_percent, path, size = None):
    resp = requests.get(url, stream=True).raw 
    
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    if image is None: 
        raise TypeError("None type") 
    
    if size == None: 
        width = int(image.shape[1] * scale_percent / 100)
        height = int(image.shape[0] * scale_percent / 100)
        dsize = (width, height)
    else: 
        dsize = size 
    
    output = cv2.resize(image, dsize)

    return cv2.imwrite(path, output)



def make_dataset_cv(dataset_name, datasource, iteration = None):
    problems_of_idx = []
    result = [] 
    iterator = 0 
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('yolov3_fashion_detection', "custom_inputs")
        path = os.path.join(base, path) 
        path = os.path.join(path, dataset_name)
        if not os.path.isdir(path):
            os.mkdir(path)
        
        # ../data/dataset_name/
        for img_index, img_row in datasource.iterrows():
                try:
                    tmp = os.path.join(path, img_row["UrunGrubuAck"])
                    if not os.path.isdir(tmp): 
                        os.mkdir(tmp) 
                    
                    if save_image_cv(img_row['resim_adresi'], 75, os.path.join(tmp, str(img_index) + '.jpg'), size=(cfg.TEST.INPUT_SIZE, cfg.TEST.INPUT_SIZE)):
                        result.append((os.path.join(tmp, str(img_index) + '.jpg'), img_row["UrunGrubuAck"]))
                    else:
                        problems_of_idx.append(img_index)

                    if iteration is not None and iterator == iteration:
                        break
                    else:
                        iterator+=1
                        if iterator % 1000 == 0: 
                            print(f"Iteration {iterator}") 
                            
                except Exception as e2:
                    problems_of_idx.append(img_index)
                    print(e2)
                    continue
    except Exception as e:
        print('An exception occurred.', e)
    return problems_of_idx, result

In [23]:
""" errors = make_dataset_cv(
    "test_object_det_with_paths", 
    test_actual
)  """

In [26]:
test_paths = errors[1]

In [29]:
test_paths[:5]

[('c:\\Users\\melih.kacaman\\projects_analytics\\yolov3\\yolov3_fashion_detection\\custom_inputs\\test_object_det_with_paths\\T SHIRT\\0.jpg',
  'T SHIRT'),
 ('c:\\Users\\melih.kacaman\\projects_analytics\\yolov3\\yolov3_fashion_detection\\custom_inputs\\test_object_det_with_paths\\T SHIRT\\1.jpg',
  'T SHIRT'),
 ('c:\\Users\\melih.kacaman\\projects_analytics\\yolov3\\yolov3_fashion_detection\\custom_inputs\\test_object_det_with_paths\\T SHIRT\\2.jpg',
  'T SHIRT'),
 ('c:\\Users\\melih.kacaman\\projects_analytics\\yolov3\\yolov3_fashion_detection\\custom_inputs\\test_object_det_with_paths\\T SHIRT\\3.jpg',
  'T SHIRT'),
 ('c:\\Users\\melih.kacaman\\projects_analytics\\yolov3\\yolov3_fashion_detection\\custom_inputs\\test_object_det_with_paths\\T SHIRT\\4.jpg',
  'T SHIRT')]

# Prediction on Test Dataset

In [49]:
def group_predict_on_custom_inputs(dataset_name:str, paths_of_images:list ,model: tf.keras.Model):  
    base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
    path = os.path.join('yolov3_fashion_detection', "custom_inputs")   
    path = os.path.join(base, path) # ./yolov3_fashion_detection/custom_inputs
    path = os.path.join(path, dataset_name) # ./yolov3_fashion_detection/custom_inputs/dataset_name 

    if len(paths_of_images) == 0:
        print("paths_of_images length is 0") 
        return

    if not os.path.isdir(path):
        print("There is not such a directory.")
        return

    for idx, tpl in enumerate(paths_of_images):
        inference(
            model=model,
            dataset_name=dataset_name,
            path_to_image=tpl[0],
            cls=tpl[1],
            config=cfg,
            SCORE_THRESHOLD=0.4
        )

        if idx % 100 == 0:
            print(f"{idx} is done.")

In [ ]:
group_predict_on_custom_inputs(
    dataset_name="test_object_det_with_paths",
    paths_of_images=test_paths,
    model=model
)

# Prediction with Iterative Method 

In [82]:
# bboxes: [x_min, y_min, x_max, y_max, probability, cls_id] format coordinates.
# class_ind = int(bbox[5]) 
# classes[class_ind]
def inference_iterative(model: tf.keras.models.Model, path_to_image: str, dataset_name: str, cls: str, config: cfg, SCORE_THRESHOLD = 0.7, IOU_THRESHOLD = 0.5,
              classes=classes, map_classes=map_classes): 
    INPUT_SIZE = config.TEST.INPUT_SIZE

    image = cv2.imread(path_to_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_size = image.shape[:2]
    image_data = utils.image_preporcess(np.copy(image), [INPUT_SIZE, INPUT_SIZE])
    image_data = image_data[np.newaxis, ...].astype(np.float32) # (1, width, height, 3)

    pred_bbox = model.predict(image_data)
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    
    while(SCORE_THRESHOLD > 0.1):
        # SCORE THRESHOLD 
        bboxes = utils.postprocess_boxes(pred_bbox, image_size, INPUT_SIZE, SCORE_THRESHOLD)
        bboxes = utils.nms(bboxes, IOU_THRESHOLD, method='nms')

        selective_bboxes = []
        for i, bbox in enumerate(bboxes):
            class_ind = int(bbox[5])
            if classes[class_ind] == map_classes[cls]:
                selective_bboxes.append(bbox)
        
        # sırala 
        if len(selective_bboxes) > 0: 
            selective_bboxes.sort(key=lambda arr: arr[4], reverse=True) 
        else:
            SCORE_THRESHOLD = SCORE_THRESHOLD - 0.05
            continue
            
        bboxes.clear() 
        bboxes.append(selective_bboxes[0])
        break
    
    if(len(bboxes) > 0):
        image_with_detections = utils.draw_bbox(image, bboxes)
        image_with_detections = cv2.cvtColor(image_with_detections, cv2.COLOR_RGB2BGR)

        # If image size is too big then resize the image for display purposes
        display_shape = (image_size[1], image_size[0])
        if image_size[0] > 1500 or image_size[1] > 1500:
            display_shape = (int(image_size[1]/4), int(image_size[0]/4))

        image_with_detections = cv2.resize(image_with_detections, display_shape)

        ds_dir = os.path.join("./custom_detections", dataset_name + "_iterative") 
        if not os.path.isdir(ds_dir):
            os.mkdir(ds_dir)
        
        ds_dir = os.path.join(ds_dir, cls)
        if not os.path.isdir(ds_dir):
            os.mkdir(ds_dir)
        
        return cv2.imwrite(os.path.join(ds_dir, os.path.split(path_to_image)[1]), image_with_detections)
    else:
        return None

In [84]:
inference_iterative(
    model=model,
    path_to_image=test_paths[200][0], 
    dataset_name="deneme", 
    cls=test_paths[200][1],
    config=cfg
)

1/1 [==============================] - 0s 286ms/step


True

In [ ]:
test_paths

In [83]:
def group_predict_on_custom_inputs_iterative(dataset_name:str, paths_of_images:list ,model: tf.keras.Model,
                                   classes=classes, map_classes=map_classes):  
    base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
    path = os.path.join('yolov3_fashion_detection', "custom_inputs")   
    path = os.path.join(base, path) # ./yolov3_fashion_detection/custom_inputs
    path = os.path.join(path, dataset_name) # ./yolov3_fashion_detection/custom_inputs/dataset_name 

    if len(paths_of_images) == 0:
        print("paths_of_images length is 0") 
        return

    if not os.path.isdir(path):
        print("There is not such a directory.")
        return

    for idx, tpl in enumerate(paths_of_images):
        inference_iterative(
            model=model,
            dataset_name=dataset_name,
            path_to_image=tpl[0],
            cls=tpl[1],
            config=cfg,
            SCORE_THRESHOLD=0.4,
            classes=classes,
            map_classes=map_classes
        )

        if idx % 100 == 0:
            print(f"{idx} is done.")

In [85]:
group_predict_on_custom_inputs_iterative(
    dataset_name="test_object_det_with_paths", 
    paths_of_images=test_paths,
    model=model
)

1/1 [==============================] - 0s 259ms/step
0 is done.
1/1 [==============================] - 0s 425ms/step
100 is done.
1/1 [==============================] - 0s 459ms/step
200 is done.
1/1 [==============================] - 1s 504ms/step
300 is done.
1/1 [==============================] - 1s 501ms/step
400 is done.
1/1 [==============================] - 0s 331ms/step
500 is done.
1/1 [==============================] - 0s 469ms/step


In [75]:
'TAKIM ELBISE' == 'TAKIM ELBISE'

True

In [81]:
map_classes

{'CEKET': 'top',
 'DENIM ETEK': 'bottom',
 'DENIM GOMLEK UZUNKOL': 'top',
 'DOKUMA ETEK': 'bottom',
 'GOMLEK KISAKOL': 'top',
 'PANTOLON': 'bottom',
 'T SHIRT': 'top',
 'TAKIM ELBISE': 'bottom'}